In [4]:
import ProximalOPT
import HD
using ArrayViews

In [7]:
reload("ProximalOPT")

In [5]:
n = 100
p = 10
X = randn(n, p)
Y = X * ones(p) + 0.1 * randn(n)
XX = X' * X / n
Xy = X' * Y / n

Abn = -Xy

groups=Array(Array{Int64, 1}, 2)
groups[1] = [1:5]
groups[2] = [6:10];

In [6]:
lambda_max = maximum(map((x)->norm(Xy[x]), groups)) + 0.1
lambda = [lambda_max, lambda_max]
beta = zeros(p)
HD.group_lasso!(beta, XX, Xy, groups, lambda)

In [9]:
h_beta = zeros(p)

groups = Array(UnitRange{Int64}, 2)
groups[1] = 1:5
groups[2] = 6:10

tic()
f = ProximalOPT.quad_f(view(XX, :, :), view(Abn, :))
g = ProximalOPT.prox_l1l2(groups, lambda)
tr = ProximalOPT.prox_grad!(h_beta, f, g; beta=0.9, show_trace = false, ABSTOL=1e-4)
prox_res = (copy(h_beta), tr, toc());

elapsed time: 0.002011532 seconds


In [11]:
n = 500
p = 1000
X = randn(n, p)
Y = X[:,1:20] * [ones(5), -ones(5), ones(5), -ones(5)] + 0.1 * randn(n)

XX = X' * X / n
Xy = X' * Y / n
Xyn = -X' * Y / n


In [26]:
lambda = 0.02 * ones(200)
groups=Array(Array{Int64, 1}, 200)
for i=1:200
    groups[i] = range((i-1)*5+1,5)
end
beta = zeros(p)
@time HD.group_lasso!(beta, XX, Xy, groups, lambda)

elapsed time: 0.001677657 seconds (954168 bytes allocated)


In [27]:
h_beta = zeros(p)

groups=Array(UnitRange{Int64}, 200)
for i=1:200
    groups[i] = range((i-1)*5+1,5)
end

tic()
f = ProximalOPT.quad_f(view(XX, :, :), view(Xyn, :))
g = ProximalOPT.prox_l1l2(groups, lambda)
@time tr = ProximalOPT.prox_grad!(h_beta, f, g; beta=0.9, show_trace = false, ABSTOL=1e-5)
prox_res = (copy(h_beta), tr, toc());

elapsed time: 0.062056914 seconds (2722016 bytes allocated)
elapsed time: 0.065889177 seconds


In [28]:
maximum(abs((beta - h_beta)))

0.002772314487584082